<a href="https://colab.research.google.com/github/hafizapatel04-bit/week3-Solar-Power-Prediction/blob/main/WEEK3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import files

# First set of files
uploaded1 = files.upload()

# Second set of files
uploaded2 = files.upload()


Saving Plant_1_Weather_Sensor_Data (1).csv.txt to Plant_1_Weather_Sensor_Data (1).csv (1).txt


Saving Cleaned_Plant_Generation_Data (2).csv to Cleaned_Plant_Generation_Data (2).csv


In [ ]:
import os
os.listdir()


['.config', 'app.py', 'sample_data']

In [ ]:
import os
os.listdir()


['.config',
 'Plant_1_Weather_Sensor_Data (1).csv.txt',
 'Plant_1_Weather_Sensor_Data (1).csv (1).txt',
 'app.py',
 'log.txt',
 'processed.csv',
 'Cleaned_Plant_Generation_Data (2).csv',
 'sample_data']

In [ ]:
# Load generation data
df_gen = pd.read_csv("Cleaned_Plant_Generation_Data (2).csv")

# Load weather data
df_weather = pd.read_csv("Plant_1_Weather_Sensor_Data (1).csv.txt")

# Check top rows
print(df_gen.head())
print(df_weather.head())



          DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  \
0  15-05-2020 00:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
1  15-05-2020 00:00   4135001  1IF53ai7Xc0U56Y       0.0       0.0   
2  15-05-2020 00:00   4135001  3PZuoBAID5Wc2HD       0.0       0.0   
3  15-05-2020 00:00   4135001  7JYdWkrLSPkdwr4       0.0       0.0   
4  15-05-2020 00:00   4135001  McdE0feGgRqW7Ca       0.0       0.0   

   DAILY_YIELD  TOTAL_YIELD  Unnamed: 7  Unnamed: 8  Unnamed: 9  ...  \
0          0.0    6259559.0         NaN         NaN         NaN  ...   
1          0.0    6183645.0         NaN         NaN         NaN  ...   
2          0.0    6987759.0         NaN         NaN         NaN  ...   
3          0.0    7602960.0         NaN         NaN         NaN  ...   
4          0.0    7158964.0         NaN         NaN         NaN  ...   

   Unnamed: 15  Unnamed: 16  Unnamed: 17  Unnamed: 18  Unnamed: 19  \
0          NaN          NaN          NaN          NaN          NaN   
1     

In [ ]:
# Make sure timestamp columns exist in both datasets and convert to datetime
df_gen['DATE_TIME'] = pd.to_datetime(df_gen['DATE_TIME'])
df_weather['DATE_TIME'] = pd.to_datetime(df_weather['DATE_TIME'])

# Merge on DATE_TIME
df = pd.merge(df_gen, df_weather, on='DATE_TIME', how='inner')
print(df.head())


   DATE_TIME  PLANT_ID_x     SOURCE_KEY_x  DC_POWER  AC_POWER  DAILY_YIELD  \
0 2020-05-15     4135001  1BY6WEcLGh8j5v7       0.0       0.0          0.0   
1 2020-05-15     4135001  1IF53ai7Xc0U56Y       0.0       0.0          0.0   
2 2020-05-15     4135001  3PZuoBAID5Wc2HD       0.0       0.0          0.0   
3 2020-05-15     4135001  7JYdWkrLSPkdwr4       0.0       0.0          0.0   
4 2020-05-15     4135001  McdE0feGgRqW7Ca       0.0       0.0          0.0   

   TOTAL_YIELD  Unnamed: 7  Unnamed: 8  Unnamed: 9  ...  Unnamed: 20  \
0    6259559.0         NaN         NaN         NaN  ...          NaN   
1    6183645.0         NaN         NaN         NaN  ...          NaN   
2    6987759.0         NaN         NaN         NaN  ...          NaN   
3    7602960.0         NaN         NaN         NaN  ...          NaN   
4    7158964.0         NaN         NaN         NaN  ...          NaN   

   Unnamed: 21  Unnamed: 22  Unnamed: 23  Unnamed: 24  PLANT_ID_y  \
0          NaN          NaN  

/tmp/ipython-input-2802145956.py:2: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_gen['DATE_TIME'] = pd.to_datetime(df_gen['DATE_TIME'])


In [ ]:
# Extract time-based features
df['hour'] = df['DATE_TIME'].dt.hour
df['weekday'] = df['DATE_TIME'].dt.weekday
df['month'] = df['DATE_TIME'].dt.month

# Rolling features (3, 6, 12 hour moving average of DC_POWER)
df['rolling_power_3'] = df['DC_POWER'].rolling(window=3, min_periods=1).mean()
df['rolling_power_6'] = df['DC_POWER'].rolling(window=6, min_periods=1).mean()
df['rolling_power_12'] = df['DC_POWER'].rolling(window=12, min_periods=1).mean()

# Optional: Lag features (previous hour power)
df['lag_1'] = df['DC_POWER'].shift(1).fillna(method='bfill')
df['lag_2'] = df['DC_POWER'].shift(2).fillna(method='bfill')

# Optional: Weather ratios (like irradiance to temperature ratio)
if 'IRRADIATION' in df.columns and 'AMBIENT_TEMPERATURE' in df.columns:
    df['irradiance_temp_ratio'] = df['IRRADIATION'] / (df['AMBIENT_TEMPERATURE'] + 1e-6)

print(df.head())


   DATE_TIME  PLANT_ID_x     SOURCE_KEY_x  DC_POWER  AC_POWER  DAILY_YIELD  \
0 2020-05-15     4135001  1BY6WEcLGh8j5v7       0.0       0.0          0.0   
1 2020-05-15     4135001  1IF53ai7Xc0U56Y       0.0       0.0          0.0   
2 2020-05-15     4135001  3PZuoBAID5Wc2HD       0.0       0.0          0.0   
3 2020-05-15     4135001  7JYdWkrLSPkdwr4       0.0       0.0          0.0   
4 2020-05-15     4135001  McdE0feGgRqW7Ca       0.0       0.0          0.0   

   TOTAL_YIELD  Unnamed: 7  Unnamed: 8  Unnamed: 9  ...  IRRADIATION  hour  \
0    6259559.0         NaN         NaN         NaN  ...          0.0     0   
1    6183645.0         NaN         NaN         NaN  ...          0.0     0   
2    6987759.0         NaN         NaN         NaN  ...          0.0     0   
3    7602960.0         NaN         NaN         NaN  ...          0.0     0   
4    7158964.0         NaN         NaN         NaN  ...          0.0     0   

   weekday  month  rolling_power_3  rolling_power_6  rolling_p

/tmp/ipython-input-2180775400.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['lag_1'] = df['DC_POWER'].shift(1).fillna(method='bfill')
/tmp/ipython-input-2180775400.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['lag_2'] = df['DC_POWER'].shift(2).fillna(method='bfill')


In [ ]:
!pip install streamlit plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.2 MB/s eta 0:00:00


In [ ]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 122.4 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("35blxlfA4Ukgk6Jc8QGDK4X2Vwy_FXW6PQkQSwhC2giR9drf")


In [ ]:
from google.colab import files

# First set of files
uploaded1 = files.upload()

# Second set of files
uploaded2 = files.upload()


Saving Cleaned_Plant_Generation_Data (2).csv to Cleaned_Plant_Generation_Data (2).csv


Saving Plant_1_Weather_Sensor_Data.csv to Plant_1_Weather_Sensor_Data.csv


In [ ]:
import os
os.listdir()


['.config',
 'app.py',
 'Plant_1_Weather_Sensor_Data.csv',
 'Cleaned_Plant_Generation_Data (2).csv',
 'sample_data']

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

# ---------------- CONFIG ----------------
st.set_page_config(page_title="Solar Power Dashboard", layout="wide")

# ---------------- DARK THEME CSS ----------------
st.markdown("""
<style>
.main {background-color: #111111;}
h1, h2, h3, h4, h5, h6, p, div, span {color: #e8e8e8 !important;}
div[data-testid="stDataFrame"] {background-color: #1b1b1b !important;}
#MainMenu, footer {visibility: hidden;}
</style>
""", unsafe_allow_html=True)

# ---------------- TITLE ----------------
st.title("⚡ Solar Power Dashboard — Plant Generation + Weather Data")

# ---------------- LOAD DATA ----------------
@st.cache_data
def load_data():
    gen = pd.read_csv("/content/Cleaned_Plant_Generation_Data (2).csv")
    weather = pd.read_csv("/content/Plant_1_Weather_Sensor_Data.csv")

    gen["DATE_TIME"] = pd.to_datetime(gen["DATE_TIME"])
    weather["DATE_TIME"] = pd.to_datetime(weather["DATE_TIME"])

    # Merge on DATE_TIME
    df = pd.merge(gen, weather, on="DATE_TIME", how="inner")

    # Create hour column
    df["hour"] = df["DATE_TIME"].dt.hour

    return df

df = load_data()

# ---------------- PREVIEW ----------------
st.header("Dataset Preview")
st.dataframe(df.head(), use_container_width=True)

# ---------------- IMPORTANT COLUMNS ----------------
important_cols = [
    "DATE_TIME",
    "AC_POWER",
    "DC_POWER" if "DC_POWER" in df.columns else None,
    "IRRADIATION" if "IRRADIATION" in df.columns else None,
    "MODULE_TEMPERATURE" if "MODULE_TEMPERATURE" in df.columns else None,
    "AMBIENT_TEMPERATURE" if "AMBIENT_TEMPERATURE" in df.columns else None,
]
important_cols = [c for c in important_cols if c]

# ---------------- LAYOUT GRAPHS ----------------
st.header("Solar Plant Performance Insights")
col1, col2 = st.columns(2)

# AC POWER over time
with col1:
    st.subheader("AC Power Over Time")
    fig1 = px.line(df, x="DATE_TIME", y="AC_POWER", template="plotly_dark")
    st.plotly_chart(fig1, use_container_width=True)

# Irradiation over time
with col2:
    if "IRRADIATION" in df.columns:
        st.subheader("Irradiation Over Time")
        fig2 = px.line(df, x="DATE_TIME", y="IRRADIATION", template="plotly_dark")
        st.plotly_chart(fig2, use_container_width=True)
    else:
        st.warning("IRRADIATION column not found.")

# Temperature charts
col3, col4 = st.columns(2)
with col3:
    if "MODULE_TEMPERATURE" in df.columns:
        st.subheader("Module Temperature Over Time")
        fig3 = px.line(df, x="DATE_TIME", y="MODULE_TEMPERATURE", template="plotly_dark")
        st.plotly_chart(fig3, use_container_width=True)

with col4:
    if "AMBIENT_TEMPERATURE" in df.columns:
        st.subheader("Ambient Temperature Over Time")
        fig4 = px.line(df, x="DATE_TIME", y="AMBIENT_TEMPERATURE", template="plotly_dark")
        st.plotly_chart(fig4, use_container_width=True)

# ---------------- POWER vs HOUR ----------------
st.header("Power vs Hour of Day")
fig5 = px.scatter(df, x="hour", y="AC_POWER", template="plotly_dark")
st.plotly_chart(fig5, use_container_width=True)

# ---------------- ML PREDICTION SECTION ----------------
st.header("🔮 AC Power Prediction Model")

# Feature selection for ML
features = []
if "IRRADIATION" in df.columns:
    features.append("IRRADIATION")
if "MODULE_TEMPERATURE" in df.columns:
    features.append("MODULE_TEMPERATURE")
if "AMBIENT_TEMPERATURE" in df.columns:
    features.append("AMBIENT_TEMPERATURE")
features.append("hour")

# Train simple linear model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = df[features]
y = df["AC_POWER"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

# ML UI Inputs
st.subheader("Predict AC Power Using Weather Inputs")
colA, colB, colC, colD = st.columns(4)

irr = colA.number_input("Irradiation", value=float(df["IRRADIATION"].mean()) if "IRRADIATION" in df.columns else 0.0)
mod_temp = colB.number_input("Module Temperature", value=float(df["MODULE_TEMPERATURE"].mean()) if "MODULE_TEMPERATURE" in df.columns else 25.0)
amb_temp = colC.number_input("Ambient Temperature", value=float(df["AMBIENT_TEMPERATURE"].mean()) if "AMBIENT_TEMPERATURE" in df.columns else 25.0)
hour_val = colD.slider("Hour of Day", 0, 23, 12)

# Predict button
if st.button("Predict AC Power"):
    import numpy as np
    inp = np.array([[irr, mod_temp, amb_temp, hour_val]])
    pred = model.predict(inp)[0]
    st.success(f"Predicted AC Power: **{pred:.2f} W**")


# ---------------- CORRELATION ----------------
st.header("Correlation Matrix (Important Features)")
corr = df[important_cols].corr()
st.dataframe(corr, use_container_width=True)

# Heatmap
fig6 = px.imshow(corr, text_auto=True, color_continuous_scale="Blues")
st.plotly_chart(fig6, use_container_width=True)


Overwriting app.py


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://angelika-uncombed-undistractingly.ngrok-free.dev" -> "http://localhost:8501">

In [ ]:
!streamlit run app.py &>/dev/null&


In [ ]:
import os
os.listdir()


['.config', 'app.py', 'sample_data']